In [1]:
import torch
import pfhedge.autogreek as autogreek
from pfhedge.instruments import BrownianStock, EuropeanOption
from pfhedge.nn import Hedger, WhalleyWilmott

# Set random seed for reproducibility
torch.manual_seed(42)

# Create a European option on a Brownian stock
stock = BrownianStock(cost=1e-4)
option = EuropeanOption(stock).to(torch.float64)

# Create a hedger using the Whalley-Wilmott model
model = WhalleyWilmott(option)
hedger = Hedger(model, model.inputs()).to(torch.float64)

# Define the pricer function
def pricer(spot, volatility, time_to_maturity):
    return hedger.price(
        option,
        init_state=(spot,),
        volatility=volatility,
        time_to_maturity=time_to_maturity,
        enable_grad=True
    )

# Set up parameters for Greek calculations
spot = torch.tensor(1.0, dtype=torch.float64)
volatility = torch.tensor(0.2, dtype=torch.float64)
time_to_maturity = torch.tensor(1.0, dtype=torch.float64)

# Calculate Greeks using autogreek
delta = autogreek.delta(pricer, spot=spot, volatility=volatility, time_to_maturity=time_to_maturity)
gamma = autogreek.gamma(pricer, spot=spot, volatility=volatility, time_to_maturity=time_to_maturity)
vega = autogreek.vega(pricer, spot=spot, volatility=volatility, time_to_maturity=time_to_maturity)

print("Calculated Greeks using autogreek:")
print(f"Delta: {delta.item():.6f}")
print(f"Gamma: {gamma.item():.6f}")
print(f"Vega: {vega.item():.6f}")

# Simulate Greeks using finite differences
epsilon = 1e-6

# Simulated Delta
price_up = pricer(spot + epsilon, volatility, time_to_maturity)
price_down = pricer(spot - epsilon, volatility, time_to_maturity)
simulated_delta = (price_up - price_down) / (2 * epsilon)

# Simulated Gamma
simulated_gamma = (price_up + price_down - 2 * pricer(spot, volatility, time_to_maturity)) / (epsilon ** 2)

# Simulated Vega
price_vol_up = pricer(spot, volatility + epsilon, time_to_maturity)
price_vol_down = pricer(spot, volatility - epsilon, time_to_maturity)
simulated_vega = (price_vol_up - price_vol_down) / (2 * epsilon)

print("\nSimulated Greeks using finite differences:")
print(f"Delta: {simulated_delta.item():.6f}")
print(f"Gamma: {simulated_gamma.item():.6f}")
print(f"Vega: {simulated_vega.item():.6f}")


TypeError: price() got an unexpected keyword argument 'volatility'

: 